In [1]:
DF = sqlContext.read.format('com.databricks.spark.csv').options(delimiter=',', header='true', inferschema='true').load("dbfs:/mnt/s3/data/train_v2_flatten.csv")
#display(DF)

In [2]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
def deleteNull_castToFloat(x):
  
  if x == None:
    return 0.0
  if x == '(not set)':
    return 0
  if x == 'No':
    return 0.0
  else:
    y = len(x)
    if y>6:
      i = int(x[ : y-6])
      f = int(x[y-6:y])
    else:
      i = 0
      f = int(x)
    a = float(i)
    b = float(f)/1000000
    return a+b
udf_deleteNull_castToFloat = F.udf(deleteNull_castToFloat, T.FloatType())

def deleteNull_castToInt(x):
  
  if x == None:
    return 0
  if x == '(not set)':
    return 0
  if x == 'No':
    return 0
  else:
    return int(x)
udf_deleteNull_castToInt = F.udf(deleteNull_castToInt, T.IntegerType())

def deleteNull_castToInt_visitNumber(x):
  
  if x == None:
    return 0
  if x == '(not set)':
    return 0
  if x == 'No':
    return 0
  if len(x)>10:
    return 0
  else:
    return int(x)
udf_deleteNull_castToInt_visitNumber = F.udf(deleteNull_castToInt_visitNumber, T.IntegerType())

In [3]:
from pyspark.sql.functions import col
DF_select = DF.select(udf_deleteNull_castToFloat("totals_transactionRevenue").alias("revenue"), "device_operatingSystem", "device_browser",  "geoNetwork_country", "channelGrouping", (col('visitStartTime')).cast("string").alias("startTime_cast"), (col('date')).cast("string").alias("date_cast"), "fullVisitorId", udf_deleteNull_castToInt("totals_hits").alias("hits"), udf_deleteNull_castToInt("totals_pageviews").alias("pageview"), udf_deleteNull_castToInt_visitNumber("visitNumber").alias("visitNumber"),
                     "geoNetwork_networkDomain","geoNetwork_region" )

In [4]:
from pyspark.sql.functions import unix_timestamp, to_date
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import month, dayofweek, hour, year, weekofyear, dayofyear

DF_select = DF_select.withColumn('date_cast', to_date('date_cast', 'yyyyMMdd'))
#DF_select = DF_select.withColumn("month", month("date_cast"))
#DF_select = DF_select.withColumn("weekday", dayofweek("date_cast"))
#DF_select = DF_select.withColumn("year", year("date_cast"))
DF_select = DF_select.withColumn("yearweek", weekofyear("date_cast"))
DF_select = DF_select.withColumn("yearday", dayofyear("date_cast"))
#DF_select = DF_select.withColumn("startTime_timestamp", from_unixtime("startTime_cast"))
#DF_select = DF_select.withColumn("hour", hour("startTime_timestamp"))
#DF_select = DF_select.drop('startTime_timestamp', "startTime_cast", 'date_cast')
DF_select.cache()

Out[ 4 ]: DataFrame[revenue: float, device_operatingSystem: string, device_browser: string, geoNetwork_country: string, channelGrouping: string, startTime_cast: string, date_cast: date, fullVisitorId: string, hits: int, pageview: int, visitNumber: int, geoNetwork_networkDomain: string, geoNetwork_region: string, yearweek: int, yearday: int]

In [5]:
DF_fea = DF_select

In [6]:
#Generating features grouped by geoNetwork_networkDomain



DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_networkDomain').sum('pageview').withColumnRenamed('sum(pageview)', 'sum_pageviews_per_network_domain'), "geoNetwork_networkDomain")

DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_networkDomain').avg('pageview').withColumnRenamed('avg(pageview)', 'avg_pageviews_per_network_domain'), "geoNetwork_networkDomain")

DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_networkDomain').agg(F.count('pageview')).withColumnRenamed('count(pageview)', 'count_pageviews_per_network_domain'),"geoNetwork_networkDomain")

DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_networkDomain').sum('hits').withColumnRenamed('sum(hits)', 'sum_hits_per_network_domain'), "geoNetwork_networkDomain")

DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_networkDomain').avg('hits').withColumnRenamed('avg(hits)', 'avg_hits_per_network_domain'), "geoNetwork_networkDomain")

DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_networkDomain').agg(F.count('hits')).withColumnRenamed('count(hits)', 'count_hits_per_network_domain'),"geoNetwork_networkDomain")

# DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_networkDomain').sum('revenue').withColumnRenamed('sum(revenue)', 'sum_revenue_per_day'), "geoNetwork_networkDomain")

# DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_networkDomain').avg('revenue').withColumnRenamed('avg(revenue)', 'avg_revenue_per_day'), "geoNetwork_networkDomain")

In [7]:
#Generating features grouped by dayofyear & weekofyear

DF_yearday = DF_fea.groupby('yearday').sum('hits').withColumnRenamed('sum(hits)', 'sum_hits_per_day').withColumnRenamed("yearday", 'yearday2')

DF_fea = DF_fea.join(DF_yearday, DF_yearday.yearday2 == DF_fea.yearday).drop('yearday2')

DF_yearday = DF_fea.groupby('yearday').avg('hits').withColumnRenamed('avg(hits)', 'avg_hits_per_day').withColumnRenamed("yearday", 'yearday2')

DF_fea = DF_fea.join(DF_yearday, DF_yearday.yearday2 == DF_fea.yearday).drop('yearday2')

DF_yearweek = DF_fea.groupby('yearweek').sum('hits').withColumnRenamed('sum(hits)', 'sum_hits_per_week').withColumnRenamed("yearweek", 'yearweek2')

DF_fea = DF_fea.join(DF_yearweek, DF_yearweek.yearweek2 == DF_fea.yearweek).drop('yearweek2')

DF_yearweek = DF_fea.groupby('yearweek').avg('hits').withColumnRenamed('avg(hits)', 'avg_hits_per_week').withColumnRenamed("yearweek", 'yearweek2')

DF_fea = DF_fea.join(DF_yearweek, DF_yearweek.yearweek2 == DF_fea.yearweek).drop('yearweek2')

# DF_fea = DF_fea.join(DF_fea.groupby('dayofyear').sum('revenue').withColumnRenamed('sum(revenue)', 'sum_revenue_per_day'), "dayofyear")

# DF_fea = DF_fea.join(DF_fea.groupby('dayofyear').avg('revenue').withColumnRenamed('avg(revenue)', 'avg_revenue_per_day'), "dayofyear")

In [8]:
#Generating features grouped by geoNetwork_region

#DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_region').sum('pageview').withColumnRenamed('sum(pageview)', 'sum_pageviews_per_region'), "geoNetwork_region")

DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_region').avg('pageview').withColumnRenamed('avg(pageview)', 'avg_pageviews_per_region'), "geoNetwork_region")

#DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_region').agg(F.count('pageview')).withColumnRenamed('count(pageview)', 'count_pageviews_per_region'),"geoNetwork_region")

#DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_region').sum('hits').withColumnRenamed('sum(hits)', 'sum_hits_per_region'), "geoNetwork_region")

DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_region').avg('hits').withColumnRenamed('avg(hits)', 'avg_hits_per_region'), "geoNetwork_region")

#DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_region').agg(F.count('hits')).withColumnRenamed('count(hits)', 'count_hits_per_region'),"geoNetwork_region")

In [9]:
#Generating features grouped by geoNetwork_country

DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_country').sum('pageview').withColumnRenamed('sum(pageview)', 'sum_pageviews_per_country'), "geoNetwork_country")

DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_country').avg('pageview').withColumnRenamed('avg(pageview)', 'avg_pageviews_per_country'), "geoNetwork_country")

DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_country').agg(F.count('pageview')).withColumnRenamed('count(pageview)', 'count_pageviews_per_country'),"geoNetwork_country")

DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_country').sum('hits').withColumnRenamed('sum(hits)', 'sum_hits_per_country'), "geoNetwork_country")

DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_country').avg('hits').withColumnRenamed('avg(hits)', 'avg_hits_per_country'), "geoNetwork_country")

DF_fea = DF_fea.join(DF_fea.groupby('geoNetwork_country').agg(F.count('hits')).withColumnRenamed('count(hits)', 'count_hits_per_country'),"geoNetwork_country")

In [10]:
#Generating features grouped by fullVisitorId

DF_fea = DF_fea.join(DF_fea.groupby('fullVisitorId').sum('pageview').withColumnRenamed('sum(pageview)', 'sum_pageviews_per_fullVisitorId'), "fullVisitorId")

#DF_fea = DF_fea.join(DF_fea.groupby('fullVisitorId').avg('pageview').withColumnRenamed('avg(pageview)', 'avg_pageviews_per_fullVisitorId'), "fullVisitorId")

#DF_fea = DF_fea.join(DF_fea.groupby('fullVisitorId').agg(F.count('pageview')).withColumnRenamed('count(pageview)', 'count_pageviews_per_fullVisitorId'),"fullVisitorId")

DF_fea = DF_fea.join(DF_fea.groupby('fullVisitorId').sum('hits').withColumnRenamed('sum(hits)', 'sum_hits_per_fullVisitorId'), "fullVisitorId")

#DF_fea = DF_fea.join(DF_fea.groupby('fullVisitorId').avg('hits').withColumnRenamed('avg(hits)', 'avg_hits_per_fullVisitorId'), "fullVisitorId")

#DF_fea = DF_fea.join(DF_fea.groupby('fullVisitorId').agg(F.count('hits')).withColumnRenamed('count(hits)', 'count_hits_per_fullVisitorId'),"fullVisitorId")



In [11]:
sqlContext.registerDataFrameAsTable(DF_fea, "DF")

DF_fea = DF_fea.join(sqlContext.sql("select fullVisitorId, sum_pageviews_per_fullVisitorId/avg_pageviews_per_region as user_pageviews_to_region from DF"), "fullVisitorId")

DF_fea = DF_fea.join(sqlContext.sql("select fullVisitorId, sum_hits_per_fullVisitorId/avg_hits_per_region as user_hits_to_region from DF"), "fullVisitorId")

In [13]:
sqlContext.sql("select fullVisitorId, visitNumber/pageview as Test from DF")

In [14]:
%sql

insert into DF select visitNumber/pageview as Test from DF

In [17]:
DF_select_group = DF_select.groupby('fullVisitorId').avg('visitNumber')

In [18]:
display(DF_select.join(DF_select_group,DF_select.fullVisitorId == DF_select_group.fullVisitorId))

In [19]:
import numpy as np
country_5k_10k = DF_select.groupBy("geoNetwork_country").count().filter("count>5000").filter("count<10000").select('geoNetwork_country')
country_list = np.array(country_5k_10k.toPandas().geoNetwork_country)

In [20]:
country_group = DF_select.groupBy("geoNetwork_country").count()
display(country_group.select(country_5k_10k.geoNetwork_country
                      

In [21]:
country_list = np.array(country_5k_10k.toPandas().geoNetwork_country)